In [1]:
import pandas as pd 
import torch_geometric as tg 
import torch 
from glob import glob
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [2]:
data = pd.read_parquet('/home/l2hebert/reddit-text/data_no_edge.parquet')

In [3]:
data

,index,body,path,link_id,subreddit,idx_path
7,0,"Herman Cain was leading too at one point, and ...",/home/l2hebert/reddit-graph/The_Donald/16632/3...,3dgpjk,The_Donald,data_0.pt
8,1,"Yeah but when he announced, I bet you dismisse...",/home/l2hebert/reddit-graph/The_Donald/16632/3...,3dgpjk,The_Donald,data_0.pt
9,2,[deleted],/home/l2hebert/reddit-graph/The_Donald/16632/3...,3dgpjk,The_Donald,data_0.pt
10,3,"You are so political savvy, maybe you should b...",/home/l2hebert/reddit-graph/The_Donald/16632/3...,3dgpjk,The_Donald,data_0.pt
11,4,"LOOOOOL\n\n**McCain**\n\n""Our economy, I think...",/home/l2hebert/reddit-graph/The_Donald/16632/3...,3dgpjk,The_Donald,data_0.pt
...,...,...,...,...,...,...
5371,5,She had a help in raising me because my parent...,/home/l2hebert/reddit-graph/AmItheAsshole/1847...,hzttv3,AmItheAsshole,data_117077.pt
10547,6,NTA. Grandma doesn't get to decide who you are...,/home/l2hebert/reddit-graph/AmItheAsshole/1847...,hzttv3,AmItheAsshole,data_117077.pt
10664,7,My family knows what’s going on with her. they...,/home/l2hebert/reddit-graph/AmItheAsshole/1847...,hzttv3,AmItheAsshole,data_117077.pt
12258,8,I'm sorry you're made out to be the antagonist...,/home/l2hebert/reddit-graph/AmItheAsshole/1847...,hzttv3,AmItheAsshole,data_117077.pt


In [3]:
folder = '/home/l2hebert/dev/processed-graphs-bert/processed/'
print(len(glob(folder + '*')))

115333


In [4]:
grouped = data.groupby('path')
print(grouped.ngroups)


117064


In [4]:
def grouped_apply(group, name):
    graph = torch.load(name)
    labels = (torch.argmax(graph.y, dim=1)).float().numpy()
    
    try:
        group['y'] = labels
        group['skip'] = 0
    except: 
        group['skip'] = 1
        group['y'] = -1

    return group

result_dfs = Parallel(n_jobs=4)(delayed(grouped_apply)(group, name) for name, group in tqdm(grouped, total=grouped.ngroups))
result = pd.concat(result_dfs)

  0%|          | 0/117064 [00:00<?, ?it/s]

In [5]:
result['skip'].value_counts()

0    2839857
Name: skip, dtype: int64

In [ ]:
skip_groups = ['c2a3ty', 'e4k2mp', 'eg09iq', 'hjn8o0', 'hm2woa', 'hujoqu', 'hwdvsq']